In [ ]:
import pandas as pd
import requests as req
import json
import time

In [3]:
page = 1
header = {"X-API-Key" : "42d38676e3881aa6c8105ddd703c301ec72faad3c6ca3fab3c93dd9fff7c3af1"}
location_raw = []
total_entry = 0
while True:
    url = f"https://api.openaq.org/v3/locations?limit=1000&page={page}"
    response = req.get(url, headers=header)
    data = response.json()
    location_meta = data.get('meta', [])
    location_data_temp = data.get('results', [])
    entry = 0
    if response.status_code == 200:
        for city in location_data_temp:
            if city['country']['id'] == 9:
                # print(f"Lcoation_id : {city['id']} City: {city.get('name')}, Sensor_id: {city['sensors'][0]['id']}")
                location_raw.append(city)
                total_entry+=1
                entry+=1
    else:
        print("Error:", response.status_code, response.text)
            
    print(f"page : {page}, found : {location_meta['found']}, Entries : {entry}, Total entries : {total_entry}")
    if type(location_meta['found']) == int:
        break
    page+=1
    time.sleep(1)

page : 1, found : >1000, Entries : 26, Total entries : 26
page : 2, found : >1000, Entries : 0, Total entries : 26
page : 3, found : >1000, Entries : 21, Total entries : 47
page : 4, found : >1000, Entries : 0, Total entries : 47
page : 5, found : >1000, Entries : 84, Total entries : 131
page : 6, found : >1000, Entries : 98, Total entries : 229
page : 7, found : >1000, Entries : 26, Total entries : 255
page : 8, found : >1000, Entries : 6, Total entries : 261
page : 9, found : >1000, Entries : 10, Total entries : 271
page : 10, found : >1000, Entries : 69, Total entries : 340
page : 11, found : >1000, Entries : 42, Total entries : 382
page : 12, found : >1000, Entries : 72, Total entries : 454
page : 13, found : >1000, Entries : 0, Total entries : 454
page : 14, found : >1000, Entries : 0, Total entries : 454
page : 15, found : >1000, Entries : 2, Total entries : 456
page : 16, found : >1000, Entries : 0, Total entries : 456
page : 17, found : >1000, Entries : 1, Total entries : 457
p

In [ ]:
location_id = []
sub_location_raw = []
cities = ['Delhi', 'Mumbai', 'Kolkata', 'Bengaluru', 'Chennai', 'Indore']
for location in location_raw:
    for city in cities:
        if city in location['name']:
            sub_location_raw.append(location)
            location_id.append(location['id'])
        

In [ ]:
location_df = []
for location in sub_location_raw:
    id = location['id']
    
    try:
        area = location['name'].split(' - ')[0].split(', ')[0]
    except:
        area = location['name'].split(' - ')[0].split(': ')[0]
        
    try:
        city = location['name'].split(' - ')[0].split(', ')[1]
    except:
        try:
            city = location['name'].split(' - ')[0].split(': ')[1]
        except:
            city = 'Mumbai'
            
    latitude = location['coordinates']['latitude']
    longitude = location['coordinates']['longitude']
    location_df.append([id, area, city, latitude, longitude])
    
location_df = pd.DataFrame(location_df, columns=['Location_id', 'Area','City', 'Latitude', 'Longitude'])
location_df.head()

,Location_id,Area,City,Latitude,Longitude
0,13,Delhi Technological University,Delhi,28.744000,77.120000
1,17,R K Puram,Delhi,28.563262,77.186937
2,50,Punjabi Bagh,Delhi,28.674045,77.131023
3,103,Income Tax Office,Delhi,28.623500,77.249400
4,235,Anand Vihar,New Delhi,28.646835,77.316032


In [65]:
sub_location_raw[34]['sensors'][0]

{'id': 15119,
 'name': 'co µg/m³',
 'parameter': {'id': 4,
  'name': 'co',
  'units': 'µg/m³',
  'displayName': 'CO mass'}}

In [56]:
sensor_df = []
listed_params = [1, 2, 3, 5, 6, 7, 8, 9]
for city in sub_location_raw:
   sensor =  city['sensors']
   for value in sensor:
        if value['parameter']['id'] in listed_params :
            location_id = city['id']
            sensor_id = value['id']
            name = value['parameter']['displayName']
            sensor_df.append([location_id, sensor_id, name])
            
sensor_df = pd.DataFrame(sensor_df, columns= ['Location_id', 'Sensor_id','Sensor_name'])
sensor_df.head()

,Location_id,Sensor_id,Sensor_name
0,13,13866,NO₂ mass
1,13,24,O₃ mass
2,13,13864,PM2.5
3,17,36,NO₂ mass
4,17,394,O₃ mass


In [53]:
sensor_df.shape

(873, 3)

In [57]:
location_sensors_data = location_df.merge(sensor_df, how = 'left', on = ['Location_id','Location_id'])
location_sensors_data.head()

,Location_id,Area,City,Latitude,Longitude,Sensor_id,Sensor_name
0,13,Delhi Technological University,Delhi,28.744000,77.120000,13866,NO₂ mass
1,13,Delhi Technological University,Delhi,28.744000,77.120000,24,O₃ mass
2,13,Delhi Technological University,Delhi,28.744000,77.120000,13864,PM2.5
3,17,R K Puram,Delhi,28.563262,77.186937,36,NO₂ mass
4,17,R K Puram,Delhi,28.563262,77.186937,394,O₃ mass


In [63]:
location_sensors_data = location_sensors_data[['Location_id', 'Sensor_id', 'Area', 'City', 'Sensor_name','Latitude', 'Longitude']]